In [2]:
import warnings
warnings.filterwarnings('ignore')

## Import the dataset

In [7]:
import sqlite3
import pandas as pd

In [4]:
import os

In [5]:
os.listdir()

['tutorial_ipynbs',
 '.gitignore.swp',
 '.gitignore',
 'database.sqlite',
 '.ipynb_checkpoints',
 'KNN_Amazon_fine_food_dataset.ipynb']

In [6]:
db_connection = sqlite3.connect('database.sqlite')

In [8]:
polarisable_dataset = pd.read_sql_query('select * from reviews where Score != 3', db_connection)

In [9]:
polarisable_dataset.columns

Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'],
      dtype='object')

In [17]:
polarisable_dataset.head(3)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...


In [18]:
df = polarisable_dataset # just to make typing easier

In [21]:
sampled_df = df.sample(n = 5000, axis = 0, random_state = 0) # random sampling of dataset

In [22]:
sampled_df.shape

(5000, 10)

## Replacing the ratings with 0 (for negative reviews) and 1 (for positive reviews).
#### Score of >3 has been considered as positive and a score of <3 has been taken as negative

In [41]:
type(sampled_df['Score'])

pandas.core.series.Series

In [37]:
scores = sampled_df['Score']

In [38]:
scores[6:12]

161937    5
451123    5
401436    5
396416    5
449554    1
379099    1
Name: Score, dtype: int64

In [39]:
scores = list(map(lambda x: 0 if x<3 else 1, scores))

In [40]:
scores[6:12]

[1, 1, 1, 1, 0, 0]

In [42]:
sampled_df['Score'] = scores

In [44]:
type(sampled_df['Score'].head(2))

pandas.core.series.Series

### Data preprocessing

##### 1. Deduplication
If a user id has multiple entries for the same timestamp, then it should be removed because it is likely that multiple entries at the same timestamp were for the same product of different variety which has a different product id than other variants


In [45]:
sampled_df.duplicated(subset = ['UserId', 'Time']).sum()

62

In [47]:
sampled_deduplicated_df = sampled_df.drop_duplicates(subset = ['UserId', 'Time'], inplace = False, keep = 'first')

#### 2. Extracting the data needed (corpus)
#### And removing html and punctuations

In [48]:
corpus = sampled_deduplicated_df['Text']

In [50]:
# dataset cleaners

import re

def remove_html(sentence):
    html_tag_re_obj = re.compile('<.*>?')
    return re.sub(html_tag_re_obj, ' ', sentence)

def remove_punctuations(sentence):
    cleaned_sentence = re.sub(r'[^a-zA-Z]', r' ', sentence)
    return cleaned_sentence

In [51]:
cleaned_corpus = []
for doc in corpus:
    cleaned_doc_1 = remove_html(doc)
    cleaned_doc_2 = remove_punctuations(doc)
    cleaned_corpus.append(cleaned_doc_2)

#### 3. Removing stop words

In [56]:
## Since the negative food reviews are likely to contain words like "don't", "didn't", etc that impart important
## meaning to the review, we check if such words exist in the corpus that we have. If these words are in the corpus,
## then they should not be in the list of stop words that we use for removing the stopwords from our corpus

count = 0
for doc in cleaned_corpus:
    if "not" in doc:
        count += 1

print(count)

count = 0
for doc in cleaned_corpus:
    if "don't" in doc:
        count += 1

print(count)

count = 0
for doc in cleaned_corpus:
    if "didn't" in doc:
        count += 1

print(count)

1890
0
0


In [57]:
from nltk.corpus import stopwords

In [58]:
stopwords = stopwords.words('english')

In [59]:
stopwords = set(stopwords)

In [60]:
stopwords.remove('not')

In [61]:
'not' in stopwords

False

In [62]:
a = [1,2,3,0,1,0,5]

In [103]:
# filtered_corpus = corpus with docs having no stop words
# doing with the sexy lambda expression

filtered_corpus = list(map(lambda doc: ' '.join(list(filter(lambda word: True if word not in stopwords else False\
                                                            , doc.split()))), corpus))

In [104]:
len(filtered_corpus)

4938

In [105]:
filtered_corpus[:2]

["We pretty much given GF pasta restaurant. It's really good disintegrate toss sauce. My celiac husband thrilled! FYI, first ingredient CORN, quinoa second, not QUITE nutritious might think -- cares!",
 'I bought first SproutMaster years ago Dowling Orchard Market passing Banning, California (back sold them).<br /><br />Through bit pricey plastic tray, I cannot complain ease use quality sprouts grown. The divider IS helpful grow different sprouts, BUT helpful grow type sprout two different stages growth. This way, always fresh sprouts handy anytime.<br /><br />They grow fast Sprout Master makes simple care them.']

In [94]:
### classical way of removing the lambda expressions
### verified the output of lambda expression output with the output of following implementation, outputs are same
# docs_without_stop_words = []
# for i, doc in enumerate(corpus):
#     non_stop_words_in_doc = []
#     for word in doc.split():
#         if word not in stopwords:
#             non_stop_words_in_doc.append(word)
            
    
#     docs_without_stop_words.append(' '.join(non_stop_words_in_doc))

#### 4. Stemming the words (SnowballStemmer)

In [97]:
from nltk.stem import SnowballStemmer

In [98]:
stemmer = SnowballStemmer('english')

In [106]:
stemmed_filtered_corpus = list(map(lambda doc: ' '.join(list(map(stemmer.stem, doc.split()))), corpus))

In [107]:
stemmed_filtered_corpus[:3]

["we had pretti much given up on gf pasta until we had this in a restaurant. it realli good and doesn't disintegr when you toss it with sauce. my celiac husband is thrilled! fyi, the first ingredi is corn, and quinoa is second, so it not quit as nutriti as you might think -- but who cares!",
 'i bought my first sproutmast year ago at the dowl orchard market when pass through banning, california (back when they sold them).<br /><br />through a bit pricey for a plastic tray, i cannot complain about the eas of use or the qualiti of the sprout grown. the divid is help to grow differ sprouts, but it is more help to grow the same type of sprout in two differ stage of growth. this way, you can alway have fresh sprout handi at anytime.<br /><br />they do grow fast and the sprout master make it so simpl to care for them.',
 'for bold coffe fan like us, this is absolut delici coffee. it has a complex taste, a hint of someth resembl red wine. i almost hesit to review it becaus it will probabl cau